<a href="https://colab.research.google.com/github/baranakyolif/Climate-similarity-analysis-of-Mediterranean-countries/blob/main/Climate_similarity_analysis_of_Mediterranean_countries.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu kod, bir hava durumu veri kümesini kullanarak şehirler ve ülkeler arasındaki iklim benzerliklerini analiz eden kapsamlı bir algoritmayı içermektedir. Aşağıda kodun algoritması, işlevleri ve matematiksel formülizasyonu detaylı bir şekilde açıklanmaktadır.
# Algoritmanın Genel Tanıtımı
Kod, Google Drive'dan bir CSV dosyası yükler ve şehirlerin iklim verilerini kullanarak benzerlik oranlarını hesaplar. Veriler, şehirler arası ve ülkeler arası benzerlikleri belirlemek için işlenir ve sonuçlar grafikler ile görselleştirilir. Ana bileşenler, verilerin işlenmesi, benzerliklerin hesaplanması ve grafiklerin oluşturulmasını içermektedir.

**İşlevlerin Açıklaması**

```
1.	Veri Yükleme ve Hazırlama
o	drive.mount(): Google Drive'ı bağlar.
o	pd.read_csv(): CSV dosyasını yükler.
```


2.	Benzerlik Hesaplama
o	calculate_similarity(data1, data2):

İki şehir arasındaki benzerliği hesaplar. Benzerlik, iki veri noktası arasındaki farkın, en yüksek değere bölünmesiyle elde edilir. Matematiksel formül:



```

benzerlik= (1−∣d1−d2∣max⁡(d1,d2,1)) × 100\text{benzerlik} =
\left(1 - \frac{|d1 - d2|}{\max(d1, d2, 1)}\right) \times 100benzerlik=(1−max(d1,d2,1)∣d1−d2∣)×100

Burada d1d1d1 ve d2d2d2 iki şehirden alınan bir özelliktir.
```


**3.	Şehir Verilerini İşleme**
o	process_city_data(weather_data): Şehirlerin verilerini işleyerek ülkeler arası benzerlik oranlarını hesaplar. Her şehir için benzerlik matrisleri oluşturur.


**4.	Grafik Çizimi**
o	draw_city_graph(similarity_matrix, cities, country): Belirli bir ülke içindeki şehirler arasındaki benzerlikleri gösteren bir ağ grafiği çizer.
o	draw_country_graph(similarity_matrix): Ülkeler arasındaki benzerlikleri gösteren bir ağ grafiği çizer.

**5.	Ülke Verilerini İşleme**
o	process_country_data(country_city_data): Ülkeler için ortalama iklim verilerini hesaplar.
o	calculate_country_similarity(country_averages): Ülkeler arasındaki benzerlikleri hesaplar ve sonuçları döndürür.

**6.	Tüm Şehirler Arası Benzerlikleri Hesaplama**
o	calculate_all_city_similarities(country_city_data): Tüm şehirler arasındaki benzerlik oranlarını hesaplar.


# Matematiksel Formülizasyon
1.	Benzerlik Hesaplama Formülü
o	İki veri noktası arasındaki benzerlik:


```

S(d1,d2)=1n∑i=1n(1−∣d1[i]−d2[i]∣max⁡(d1[i],d2[i],1))×100S(d1, d2) =
 \frac{1}{n} \sum_{i=1}^{n} \left(1 - \frac{|d1[i] - d2[i]|}{\max(d1[i],
  d2[i], 1)}\right) \times 100S(d1,d2)=n1i=1∑n(1−max(d1[i],
  d2[i],1)∣d1[i]−d2[i]∣)×100
```


Burada nnn özellik sayısını ifade eder.
2.	Ülke Ortalamaları
o	Bir ülkenin iklim ortalamaları:


```

Acountry=1m∑j=1mAcity,jA_{country} = \frac{1}{m} \sum_{j=1}^{m} A_{city,j}Acountry=m1j=1∑mAcity,j
```

Burada AcountryA_{country}Acountry ülke ortalaması, Acity,jA_{city,j}Acity,j j'inci şehrin özellikleri ve mmm o ülkedeki şehir sayısını ifade eder.
3.	Benzerlik Matrisleri

o	Benzerlik matrisinin her bir elemanı:


```

 Mi,j=S(Acountry,i,Acountry,j)M_{i,j} = S(A_{country,i}, A_{country,j})Mi,j=S(Acountry,i,Acountry,j)
```

Burada Mi,jM_{i,j}Mi,j iii ve jjj şehirleri arasındaki benzerliği gösterir.

# Sonuç
Bu kod, iklim verilerini analiz etmek için etkili bir yöntem sunmakta ve şehirler ile ülkeler arasındaki benzerlikleri görsel olarak sunmaktadır. Kullanıcılar, bu verileri kullanarak iklim değişkenliklerini ve benzerliklerini daha iyi anlayabilirler. Analiz sonucunda elde edilen grafikler, kullanıcıların veriler arasında bağlantılar kurmasına yardımcı olur.


In [ ]:
# @title Python kodları
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from google.colab import drive

# Google Drive'ı bağlama
drive.mount('/content/drive')

# CSV dosyasını Google Drive'dan yükleme
file_path = '/content/drive/My Drive/daily_weather_data2.csv' # Google Drive'a kaydettiğiniz csv dosyasının yolu ile değiştirin burayı
weather_data = pd.read_csv(file_path)

# Benzerlik hesaplama fonksiyonu
def calculate_similarity(data1, data2):
    similarity = 0
    for key in data1:
        if data1[key] is not None and data2[key] is not None:
            similarity += 1 - abs(data1[key] - data2[key]) / max(data1[key], data2[key], 1)
    return (similarity / len(data1)) * 100

# Şehirler arası veriyi işle
def process_city_data(weather_data):
    cities = weather_data['City'].unique()
    country_city_data = {}
    city_similarities = []
    country_similarity_matrices = {}

    for city in cities:
        city_data = weather_data[weather_data['City'] == city].iloc[0]
        country = city_data['Country']

        if country not in country_city_data:
            country_city_data[country] = {}

        country_city_data[country][city] = {
            'temperature_min': city_data['Temperature Min'],
            'temperature_max': city_data['Temperature Max'],
            'precipitation': city_data['Precipitation'],
            'windspeed': city_data['Windspeed'],
            'humidity': city_data['Humidity']
        }

    for country, cities in country_city_data.items():
        print(f"{country} Şehirler Arası Benzerlik Oranları:")
        similarity_matrix = {}
        for city1 in cities:
            similarity_matrix[city1] = {}
            for city2 in cities:
                if city1 != city2:
                    similarity = calculate_similarity(country_city_data[country][city1], country_city_data[country][city2])
                    similarity_matrix[city1][city2] = similarity
                    print(f"  {city1} - {city2}: %{similarity:.2f} benzerlik")
                    city_similarities.append(similarity)
        country_similarity_matrices[country] = similarity_matrix
        print()  # Boşluk bırak

    return country_city_data, country_similarity_matrices, city_similarities

def draw_city_graph(similarity_matrix, cities, country):
    G = nx.Graph()

    # Düğümler ekleme
    for city in cities:
        G.add_node(city)

    # Kenarları ekleme (benzerliklere göre)
    for city1 in cities:
        for city2 in cities:
            if city1 != city2:
                similarity = similarity_matrix[city1][city2]
                if similarity >= 30:  # threshold değeri
                    G.add_edge(city1, city2, weight=similarity)

    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)  # Daha stabil bir görüntü için seed kullanımı
    labels = {city: city for city in cities}
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=2000, node_color='lightblue', font_size=10, font_color='black', edge_color='grey')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    plt.title(f"{country} Şehirlerarası İklim Benzerlik Ağı")
    plt.show()

# Ülkeler arası benzerlikleri hesapla ve yazılı verileri döndür
def process_country_data(country_city_data):
    country_averages = {}

    for country, city_data in country_city_data.items():
        avg_climate = {
            'temperature_min': sum(city['temperature_min'] for city in city_data.values()) / len(city_data),
            'temperature_max': sum(city['temperature_max'] for city in city_data.values()) / len(city_data),
            'precipitation': sum(city['precipitation'] for city in city_data.values()) / len(city_data),
            'windspeed': sum(city['windspeed'] for city in city_data.values()) / len(city_data),
            'humidity': sum(city['humidity'] for city in city_data.values()) / len(city_data)
        }
        country_averages[country] = avg_climate

    return country_averages

def calculate_country_similarity(country_averages):
    country_similarity_matrix = {}

    countries_list = list(country_averages.keys())
    print("Ülkeler Arası İklim Benzerlik Oranları:")

    for i in range(len(countries_list)):
        country1 = countries_list[i]
        country_similarity_matrix[country1] = {}
        for j in range(i + 1, len(countries_list)):
            country2 = countries_list[j]
            similarity = calculate_similarity(country_averages[country1], country_averages[country2])
            country_similarity_matrix[country1][country2] = similarity
            print(f"  {country1} - {country2}: %{similarity:.2f} benzerlik")

    return country_similarity_matrix

def draw_country_graph(similarity_matrix):
    G = nx.Graph()

    # Düğümler ekleme
    for country in similarity_matrix.keys():
        G.add_node(country)

    # Kenarları ekleme
    for country1, similarities in similarity_matrix.items():
        for country2, similarity in similarities.items():
            if similarity >= 30:  # threshold değeri
                G.add_edge(country1, country2, weight=similarity)

    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)
    labels = {country: country for country in similarity_matrix.keys()}
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=3000, node_color='lightgreen', font_size=12, font_color='black', edge_color='grey')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='blue')
    plt.title("Ülkelerarası İklim Benzerlik Ağı")
    plt.show()

# Tüm ülkelerdeki şehirlerin benzerlik oranlarını hesapla
def calculate_all_city_similarities(country_city_data):
    all_cities = [(country, city, data) for country, cities in country_city_data.items() for city, data in cities.items()]
    all_city_similarities = []

    print("\nTüm Ülkelerdeki Şehirler Arası İklim Benzerlik Oranları:")
    for i in range(len(all_cities)):
        country1, city1, data1 = all_cities[i]
        for j in range(i + 1, len(all_cities)):
            country2, city2, data2 = all_cities[j]
            similarity = calculate_similarity(data1, data2)
            print(f"  {city1} ({country1}) - {city2} ({country2}): %{similarity:.2f} benzerlik")
            all_city_similarities.append(similarity)

    return all_city_similarities

# Şehir verilerini işleme ve grafiğini çizme
country_city_data, country_similarity_matrices, city_similarities = process_city_data(weather_data)
for country, cities in country_city_data.items():
    draw_city_graph(country_similarity_matrices[country], cities.keys(), country)

# Ülke verilerini işleme ve ülkeler arası benzerlik ağını çizme
country_averages = process_country_data(country_city_data)
country_similarity_matrix = calculate_country_similarity(country_averages)
draw_country_graph(country_similarity_matrix)

# Tüm ülkelerdeki şehirlerin benzerlik oranlarını yazdırma
all_city_similarities = calculate_all_city_similarities(country_city_data)


Bu işlemi uygulamadan önce herhangi bir api'den ihtiyacınız olan şehir ve ülkelerin verilerini almanız gerekmektedir. Bunun için aşağıya şöyle bir kod bırakıyorum. Bu kod belirtilen kordinatlardan (burada akdeniz bölgesi tercih edilmiştir) gerekli bilgileri alarak google drive hesabınıza csv formatında kaydeder. Üstteki kod ise bu formata göre yapılandırılmıştır ve işlemleri bu bağlamda yapar

In [ ]:
# @title apiden csvye veri çekme
import requests
import pandas as pd
from google.colab import drive
import os

# Google Drive'ı bağlayın
drive.mount('/content/drive', force_remount=True)

# API anahtarınızı buraya ekleyin
API_KEY = 'HERHNGİ BİR VERİ SAĞLAYICI (METEOROLOJİ KURUMUNUN) API ANAHTARINI ALIN VE BURAYA GİRİN BEN BURADA meteoblue APISINI TERCİH ETTİM'

# Ülkeler ve şehirler
countries = {
    'Türkiye': [
        'Antalya', 'Mersin', 'İzmir', 'Bodrum', 'Alanya'
    ],
    'Yunanistan': [
        'Atina', 'Selanik', 'Patras', 'Heraklion (Girit Adası)'
    ],
    'İtalya': [
        'Napoli', 'Genova', 'Palermo', 'Bari', 'Cagliari (Sardunya Adası)'
    ],
    'Fransa': [
        'Nice', 'Marsilya', 'Toulon', 'Cannes'
    ],
    'İspanya': [
        'Barselona', 'Valensiya', 'Malaga', 'Alicante', 'Palma de Mallorca (Mallorca Adası)'
    ],
    'İsrail': [
        'Tel Aviv', 'Hayfa', 'Aşdod'
    ],
    'Mısır': [
        'İskenderiye', 'Port Said', 'Marsa Matruh'
    ],
    'Lübnan': [
        'Beyrut', 'Sayda', 'Trablusşam'
    ],
    'Suriye': [
        'Lazkiye', 'Tartus'
    ],
    'Libya': [
        'Trablus (Tripoli)', 'Bengazi', 'Derne'
    ],
    'Tunus': [
        'Tunis', 'Sfax', 'Sousse'
    ],
    'Cezayir': [
        'Cezayir (Alger)', 'Oran', 'Annaba'
    ],
    'Fas': [
        'Tanca', 'Rabat', 'Casablanca'
    ],
    'Hırvatistan': [
        'Dubrovnik', 'Split', 'Zadar'
    ],
    'Arnavutluk': [
        'Dıraç (Durrës)', 'Vlorë', 'Sarandë'
    ],
    'Karadağ': [
        'Kotor', 'Bar', 'Herceg Novi'
    ],
    'Slovenya': [
        'Piran', 'Koper'
    ],
    'Kıbrıs': [
        'Lefkoşa', 'Girne', 'Larnaka', 'Limasol'
    ]
}

# Şehirlerin koordinatları (lat, lon)
city_coordinates = {
    'Antalya': (36.8969, 30.7133),
    'Mersin': (36.8121, 34.6415),
    'İzmir': (38.4237, 27.1428),
    'Bodrum': (37.0344, 27.4305),
    'Alanya': (36.5437, 32.0025),
    'Atina': (37.9838, 23.7275),
    'Selanik': (40.6401, 22.9444),
    'Patras': (38.2466, 21.7346),
    'Heraklion (Girit Adası)': (35.3387, 25.1442),
    'Napoli': (40.8518, 14.2681),
    'Genova': (44.4056, 8.9463),
    'Palermo': (38.1157, 13.3615),
    'Bari': (41.1171, 16.8719),
    'Cagliari (Sardunya Adası)': (39.2238, 9.1217),
    'Nice': (43.7102, 7.2620),
    'Marsilya': (43.2965, 5.3698),
    'Toulon': (43.1242, 5.9280),
    'Cannes': (43.5528, 7.0174),
    'Barselona': (41.3851, 2.1734),
    'Valensiya': (39.4699, -0.3763),
    'Malaga': (36.7213, -4.4214),
    'Alicante': (38.3452, -0.4810),
    'Palma de Mallorca (Mallorca Adası)': (39.5696, 2.6502),
    'Tel Aviv': (32.0853, 34.7818),
    'Hayfa': (32.7940, 34.9896),
    'Aşdod': (31.8014, 34.6431),
    'İskenderiye': (31.2001, 29.9187),
    'Port Said': (31.2653, 32.3019),
    'Marsa Matruh': (31.3547, 27.2373),
    'Beyrut': (33.8938, 35.5018),
    'Sayda': (33.5608, 35.3981),
    'Trablusşam': (34.4330, 35.8466),
    'Lazkiye': (35.5236, 35.7917),
    'Tartus': (34.8914, 35.8866),
    'Trablus (Tripoli)': (32.8872, 13.1913),
    'Bengazi': (32.1167, 20.0667),
    'Derne': (32.7614, 22.6367),
    'Tunis': (36.8065, 10.1815),
    'Sfax': (34.7406, 10.7603),
    'Sousse': (35.8256, 10.6084),
    'Cezayir (Alger)': (36.7372, 3.0860),
    'Oran': (35.6971, -0.6308),
    'Annaba': (36.9025, 7.7521),
    'Tanca': (35.7595, -5.8340),
    'Rabat': (34.0209, -6.8417),
    'Casablanca': (33.5731, -7.5898),
    'Dubrovnik': (42.6507, 18.0944),
    'Split': (43.5081, 16.4402),
    'Zadar': (44.1194, 15.2314),
    'Dıraç (Durrës)': (41.3231, 19.4414),
    'Vlorë': (40.4661, 19.4897),
    'Sarandë': (39.8755, 20.0057),
    'Kotor': (42.4247, 18.7712),
    'Bar': (42.0935, 19.1003),
    'Herceg Novi': (42.4525, 18.5312),
    'Piran': (45.5271, 13.5680),
    'Koper': (45.5469, 13.7294),
    'Lefkoşa': (35.1856, 33.3823),
    'Girne': (35.3403, 33.3196),
    'Larnaka': (34.9221, 33.6232),
    'Limasol': (34.7071, 33.0226)
}

# API'den günlük hava durumu verilerini çekme fonksiyonu
def fetch_daily_weather_data(lat, lon):
    url = f"https://my.meteoblue.com/packages/basic-day?lat={lat}&lon={lon}&apikey={API_KEY}&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        print(f"API Response for coordinates ({lat}, {lon}): {data}")

        # Kontrol: Verinin 'data_day' altında olup olmadığını kontrol edin
        if 'data_day' in data:
            return data['data_day']
        else:
            print(f"No daily data available for coordinates: ({lat}, {lon})")
            return None
    else:
        print(f"Error fetching data for coordinates: {lat}, {lon}, Status Code: {response.status_code}")
        return None
# Verileri işleme ve CSV dosyasına kaydetme fonksiyonu
def process_and_save_data():
    all_data = []

    for country, cities in countries.items():
        for city in cities:
            if city in city_coordinates:
                lat, lon = city_coordinates[city]
                daily_data = fetch_daily_weather_data(lat, lon)

                if daily_data:
                    for i in range(len(daily_data['time'])):
                        row = {
                            'Country': country,
                            'City': city,
                            'Time': daily_data['time'][i],
                            'Temperature Min': daily_data['temperature_min'][i],
                            'Temperature Max': daily_data['temperature_max'][i],
                            'Precipitation': daily_data['precipitation'][i],
                            'Wind Speed': daily_data['windspeed_mean'][i],  # Correct key
                            'Humidity': daily_data['relativehumidity_mean'][i]  # Assuming 'humidity' means relative humidity
                        }
                        all_data.append(row)

    # Veriyi DataFrame'e dönüştürme
    df = pd.DataFrame(all_data)

    # CSV dosyasına kaydetme
    output_file = '/content/drive/My Drive/akdenizall.csv'
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

# Veriyi işleme ve CSV dosyasına kaydetme
process_and_save_data()


EĞER BAŞKA BİR VERİ SAĞLAYICI KULLANACAKSANIZ İHTİYACINIZ OLACAK KORDİNAT BİLGİLERİNİ BURAYA BIRAKIYORUM:

# @title kordinatlar
# Şehir isimleri ve koordinatları
cities = {
    # Türkiye
    'Antalya': {'lat': 36.8969, 'lon': 30.7133},
    'Mersin': {'lat': 36.8121, 'lon': 34.6415},
    'İzmir': {'lat': 38.4237, 'lon': 27.1428},
    'Bodrum': {'lat': 37.0344, 'lon': 27.4305},
    'Alanya': {'lat': 36.5438, 'lon': 32.0003},

    # Yunanistan
    'Atina': {'lat': 37.9838, 'lon': 23.7275},
    'Selanik': {'lat': 40.6401, 'lon': 22.9444},
    'Patras': {'lat': 38.2466, 'lon': 21.7346},
    'Heraklion (Girit Adası)': {'lat': 35.3387, 'lon': 25.1442},

    # İtalya
    'Napoli': {'lat': 40.8518, 'lon': 14.2681},
    'Genova': {'lat': 44.4056, 'lon': 8.9463},
    'Palermo': {'lat': 38.1157, 'lon': 13.3615},
    'Bari': {'lat': 41.1171, 'lon': 16.8719},
    'Cagliari (Sardunya Adası)': {'lat': 39.2238, 'lon': 9.1217},

    # Fransa
    'Nice': {'lat': 43.7102, 'lon': 7.2620},
    'Marsilya': {'lat': 43.2965, 'lon': 5.3698},
    'Toulon': {'lat': 43.1242, 'lon': 5.9280},
    'Cannes': {'lat': 43.5528, 'lon': 7.0174},

    # İspanya
    'Barselona': {'lat': 41.3851, 'lon': 2.1734},
    'Valensiya': {'lat': 39.4699, 'lon': -0.3763},
    'Malaga': {'lat': 36.7213, 'lon': -4.4214},
    'Alicante': {'lat': 38.3452, 'lon': -0.4810},
    'Palma de Mallorca (Mallorca Adası)': {'lat': 39.5696, 'lon': 2.6502},

    # İsrail
    'Tel Aviv': {'lat': 32.0853, 'lon': 34.7818},
    'Hayfa': {'lat': 32.7940, 'lon': 34.9896},
    'Aşdod': {'lat': 31.8014, 'lon': 34.6431},

    # Mısır
    'İskenderiye': {'lat': 31.2001, 'lon': 29.9187},
    'Port Said': {'lat': 31.2653, 'lon': 32.3019},
    'Marsa Matruh': {'lat': 31.3547, 'lon': 27.2373},

    # Lübnan
    'Beyrut': {'lat': 33.8938, 'lon': 35.5018},
    'Sayda': {'lat': 33.5608, 'lon': 35.3981},
    'Trablusşam': {'lat': 34.4330, 'lon': 35.8466},

    # Suriye
    'Lazkiye': {'lat': 35.5236, 'lon': 35.7917},
    'Tartus': {'lat': 34.8914, 'lon': 35.8866},

    # Libya
    'Trablus (Tripoli)': {'lat': 32.8872, 'lon': 13.1913},
    'Bengazi': {'lat': 32.1167, 'lon': 20.0667},
    'Derne': {'lat': 32.7614, 'lon': 22.6367},

    # Tunus
    'Tunis': {'lat': 36.8065, 'lon': 10.1815},
    'Sfax': {'lat': 34.7406, 'lon': 10.7603},
    'Sousse': {'lat': 35.8256, 'lon': 10.6084},

    # Cezayir
    'Cezayir (Alger)': {'lat': 36.7372, 'lon': 3.0860},
    'Oran': {'lat': 35.6971, 'lon': -0.6308},
    'Annaba': {'lat': 36.9025, 'lon': 7.7521},

    # Fas
    'Tanca': {'lat': 35.7595, 'lon': -5.8340},
    'Rabat': {'lat': 34.0209, 'lon': -6.8417},
    'Casablanca': {'lat': 33.5731, 'lon': -7.5898},

    # Hırvatistan
    'Dubrovnik': {'lat': 42.6507, 'lon': 18.0944},
    'Split': {'lat': 43.5081, 'lon': 16.4402},
    'Zadar': {'lat': 44.1194, 'lon': 15.2314},

    # Arnavutluk
    'Dıraç (Durrës)': {'lat': 41.3231, 'lon': 19.4414},
    'Vlorë': {'lat': 40.4661, 'lon': 19.4897},
    'Sarandë': {'lat': 39.8755, 'lon': 20.0057},

    # Karadağ
    'Kotor': {'lat': 42.4247, 'lon': 18.7712},
    'Bar': {'lat': 42.0935, 'lon': 19.1003},
    'Herceg Novi': {'lat': 42.4525, 'lon': 18.5312},

    # Slovenya
    'Piran': {'lat': 45.5271, 'lon': 13.5680},
    'Koper': {'lat': 45.5469, 'lon': 13.7294},

    # Kıbrıs (Kuzey Kıbrıs Türk Cumhuriyeti ve Kıbrıs Cumhuriyeti)
    'Lefkoşa': {'lat': 35.1856, 'lon': 33.3823},
    'Girne': {'lat': 35.3403, 'lon': 33.3196},
    'Larnaka': {'lat': 34.9221, 'lon': 33.6232},
    'Limasol': {'lat': 34.7071, 'lon': 33.0226}
}


EĞER KURACAĞNIZ SİSTEMLERİ NASA APILERIYLE UYUMLU BIR HALE GETİRMEK İSTERSENİZ AŞAĞIDAKİ KOD GRUBUNU KULLANABİLİRSİNİZ (ÖRNEKTİR) ÖZELLİKLE FLR": "Güneş Parlaması (FLR)", "SEP": "Güneş Enerjili Parçacık (SEP)", "MPC": "Manyetopoz Geçişi (MPC)", "RBE": "Radyasyon Kuşağı Geliştirme (RBE) VERİLERİ İÇİN BU KODU EKLEDİM

APIDEN ALINABİLECEK BİLGİLER:
   "CME": "Koronal Kütle Atımı (CME)",
    "CMEAnalytics": "Koronal Kütle Atımı (CME) Analizi",
    "GST": "Jeomanyetik Fırtına (GST)",
    "IPS": "Gezegenlerarası Şok (IPS)",
    "FLR": "Güneş Parlaması (FLR)",
    "SEP": "Güneş Enerjili Parçacık (SEP)",
    "MPC": "Manyetopoz Geçişi (MPC)",
    "RBE": "Radyasyon Kuşağı Geliştirme (RBE)",
    "HSS": "Yüksek Hızlı Akış (HSS)",
    "WSAEnlilSimulations": "WSA+Enlil Simülasyonu",
    "notifications": "Bildirimler"

In [ ]:
# @title ÖRNEK KOD: NASA ASTROİD BULMA - Neows
import requests
import json

# API anahtarınızı buraya girin
api_key = "ÜCRETSİZ NASA API ANAHTARINIZI BURAYA GİRİN"

def get_neo_feed(start_date, end_date):
    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    return data

def lookup_asteroid(asteroid_id):
    url = f"https://api.nasa.gov/neo/rest/v1/neo/{asteroid_id}?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    return data

def browse_neo_data():
    url = f"https://api.nasa.gov/neo/rest/v1/neo/browse?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    return data

def print_pretty(data):
    # JSON verisini girintili ve okunabilir şekilde yazdırır
    formatted_data = json.dumps(data, indent=4, ensure_ascii=False)
    print(formatted_data)

def main():
    print("Yapmak istediğiniz işlemi seçin:")
    print("1. Belirli bir tarih aralığında asteroitleri listele (Neo - Feed)")
    print("2. Belirli bir asteroiti ara (Neo - Lookup)")
    print("3. Genel asteroit veritabanını tarama (Neo - Browse)")

    choice = input("Seçiminiz (1/2/3): ")

    if choice == '1':
        start_date = input("Başlangıç tarihi (YYYY-MM-DD): ")
        end_date = input("Bitiş tarihi (YYYY-MM-DD): ")
        data = get_neo_feed(start_date, end_date)
        print_pretty(data)

    elif choice == '2':
        asteroid_id = input("Asteroit SPK-ID'si girin: ")
        data = lookup_asteroid(asteroid_id)
        print_pretty(data)

    elif choice == '3':
        data = browse_neo_data()
        print_pretty(data)

    else:
        print("Geçersiz seçim. Lütfen 1, 2 veya 3'ü seçin.")

if __name__ == "__main__":
    main()
